In [405]:
import numpy as np
import pandas as pd
from textacy import text_stats, make_spacy_doc
from datasets import Dataset, load_dataset
import matplotlib.pyplot as plt
import itertools
import torch
import os

In [406]:
df_sample = pd.DataFrame(load_dataset("argilla/ultrafeedback-binarized-preferences-cleaned")["train"]).sample(n=15000, random_state=42)
df_clean = df_sample.copy()

In [407]:
df_clean = df_clean.reset_index(drop=True)
df_clean

,source,prompt,chosen,chosen-rating,chosen-model,rejected,rejected-rating,rejected-model
0,sharegpt,Topics: Wound management for general practitio...,[{'content': 'Topics: Wound management for gen...,5.00,wizardlm-70b,[{'content': 'Topics: Wound management for gen...,3.75,ultralm-13b
1,flan_v2_niv2,"Part 1. Definition\nIn this task, you are give...","[{'content': 'Part 1. Definition In this task,...",4.50,wizardlm-70b,"[{'content': 'Part 1. Definition In this task,...",3.25,vicuna-33b
2,sharegpt,You will act as an voice changer. You will cha...,[{'content': 'You will act as an voice changer...,4.50,wizardlm-13b,[{'content': 'You will act as an voice changer...,1.75,ultralm-13b
3,ultrachat,Write a well-researched paper on the physiolog...,[{'content': 'Write a well-researched paper on...,4.75,wizardlm-13b,[{'content': 'Write a well-researched paper on...,3.00,ultralm-13b
4,ultrachat,Create a step-by-step recipe for making fluffy...,[{'content': 'Create a step-by-step recipe for...,5.00,llama-2-7b-chat,[{'content': 'Create a step-by-step recipe for...,4.50,falcon-40b-instruct
...,...,...,...,...,...,...,...,...
14995,flan_v2_cot,"[QUESTION] If ""Two kids stepping stones over t...","[{'content': '[QUESTION] If ""Two kids stepping...",4.75,gpt-3.5-turbo,"[{'content': '[QUESTION] If ""Two kids stepping...",1.75,llama-2-7b-chat
14996,flan_v2_niv2,"Instructions: Given an object and a part, deci...",[{'content': 'Instructions: Given an object an...,4.50,vicuna-33b,[{'content': 'Instructions: Given an object an...,3.50,llama-2-7b-chat
14997,sharegpt,Write me a letter asking my friend Dan what he...,[{'content': 'Write me a letter asking my frie...,4.50,ultralm-13b,[{'content': 'Write me a letter asking my frie...,1.25,llama-2-7b-chat
14998,flan_v2_niv2,You will be given a definition of a task first...,[{'content': 'You will be given a definition o...,2.00,llama-2-70b-chat,[{'content': 'You will be given a definition o...,1.00,vicuna-33b


In [408]:
df_clean['chosen'] = df_clean['chosen'].apply(lambda x: x[1]['content'])
df_clean['rejected'] = df_clean['rejected'].apply(lambda x: x[1]['content'])

## Need to bring in the docs to filter before organizing and running stats

In [409]:
topics = pd.read_csv("UF10k_mixedbread_topics.csv")
docs = pd.read_csv("UF_mixedbread_docs.csv")

In [410]:
#Cleaning
docs['topic_id'] = docs['topic_id'].str.extract(r'(\d+)').astype(int)

docs['x_cord'] = docs['x_cord'].str.extract(r'(\d+.\d+)').astype(float)
docs['y_cord'] = docs['y_cord'].str.extract(r'(\d+.\d+)').astype(float)

In [411]:
# Merging to get all the data in one place
docs_and_topics = docs.merge(topics.drop(columns=['doc_id', 'topic_id']), on='content')
docs_and_topics = docs_and_topics.drop(columns=['term_id', 'doc_id']).rename(columns={'10': 'metadata', 'content': 'prompt'})
df_clean = df_clean.merge(docs_and_topics, on='prompt')

In [412]:
df_clean

,source,prompt,chosen,chosen-rating,chosen-model,rejected,rejected-rating,rejected-model,x_cord,y_cord,topic_id,embeddings,metadata,topic_name
0,sharegpt,Topics: Wound management for general practitio...,General practitioners play a vital role in the...,5.00,wizardlm-70b,Wound management is a crucial aspect of genera...,3.75,ultralm-13b,0.915843,0.482842,22,"[0.20282909274101257, -0.05036671459674835, -0...","('metadata', {'source': 'sharegpt', 'rating': ...",Childhood Stress and Health
1,flan_v2_niv2,"Part 1. Definition\nIn this task, you are give...",Step 1: Identify the language of the post. The...,4.50,wizardlm-70b,Non-hateful\n\nExplanation: This post seems to...,3.25,vicuna-33b,2.740021,4.862818,14,"[0.06639007478952408, 0.23666216433048248, -0....","('metadata', {'source': 'flan_v2_niv2', 'ratin...",Analyzing Product Sentiment
2,sharegpt,You will act as an voice changer. You will cha...,An umbrella has been brought by her on the way.,4.50,wizardlm-13b,"Alright, just let me know which form of voice ...",1.75,ultralm-13b,2.045614,4.058925,15,"[-0.4921853840351105, -0.1649392992258072, -0....","('metadata', {'source': 'sharegpt', 'rating': ...",Language Analysis
3,ultrachat,Write a well-researched paper on the physiolog...,Title: The Physiological and Psychological Imp...,4.75,wizardlm-13b,Title: The Physiological and Psychological Imp...,3.00,ultralm-13b,1.698340,0.061782,8,"[-0.30388227105140686, 0.15756076574325562, 0....","('metadata', {'source': 'ultrachat', 'rating':...",Healthy Lifestyle Exploration
4,ultrachat,Create a step-by-step recipe for making fluffy...,"Of course, I'd be happy to help you with that!...",5.00,llama-2-7b-chat,Here is a step-by-step recipe for making fluff...,4.50,falcon-40b-instruct,3.702293,1.716949,6,"[-0.05435372143983841, 0.28580591082572937, -0...","('metadata', {'source': 'ultrachat', 'rating':...",Culinary Creations
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10203,sharegpt,Transform is an emerging brand that delivers m...,"Welcome to Transform, your go-to source for in...",4.75,gpt-3.5-turbo,"Sure, here's a possible description for the ""A...",4.00,llama-2-70b-chat,0.732869,1.959356,21,"[-0.4771421253681183, -0.0352528840303421, -0....","('metadata', {'source': 'sharegpt', 'rating': ...",Marketing Strategy Platform
10204,evol_instruct,"Using Python and the scikit-learn library, how...",Detecting sarcasm in text is a challenging tas...,4.75,wizardlm-13b,"Hello! As a helpful and respectful assistant, ...",4.00,llama-2-13b-chat,2.532706,5.040441,14,"[0.3401801884174347, 0.21816378831863403, 0.02...","('metadata', {'source': 'evol_instruct', 'rati...",Analyzing Product Sentiment
10205,flan_v2_cot,"[QUESTION] If ""Two kids stepping stones over t...","Yes, the premise mentions pink plaid pants and...",4.75,gpt-3.5-turbo,Confidence: 80%\n\nYou have provided several s...,1.75,llama-2-7b-chat,3.797116,0.511288,18,"[0.39390242099761963, 0.10773289203643799, -0....","('metadata', {'source': 'flan_v2_cot', 'rating...",Gender and Clothing Perceptions
10206,sharegpt,Write me a letter asking my friend Dan what he...,"Dear Dan,\n\nWe hope this letter finds you wel...",4.50,ultralm-13b,I cannot fulfill your request as it goes again...,1.25,llama-2-7b-chat,0.873350,3.242940,1,"[0.7983337640762329, -0.37210243940353394, 0.5...","('metadata', {'source': 'sharegpt', 'rating': ...",Cultural Expression


## Now we need put all the responses together, regradless of whether they are rejected or accepted

In [413]:
df_reject = df_clean.iloc[:, [0 , 1, 5, 6, 7]]
df_accept = df_clean.iloc[:, [0, 1, 2, 3, 4]]

In [414]:
df_reject = df_reject.rename(columns={'rejected' : 'response', 'rejected-rating' : 'rating', 'rejected-model' : 'model'})
df_reject['chosen'] = 0
df_accept = df_accept.rename(columns={'chosen' : 'response', 'chosen-rating': 'rating', 'chosen-model' : 'model'})
df_accept['chosen'] = 1

In [415]:
frames = [df_accept, df_reject]

#put them all together

df_all = pd.concat(frames)

In [416]:
df_all

,source,prompt,response,rating,model,chosen
0,sharegpt,Topics: Wound management for general practitio...,General practitioners play a vital role in the...,5.00,wizardlm-70b,1
1,flan_v2_niv2,"Part 1. Definition\nIn this task, you are give...",Step 1: Identify the language of the post. The...,4.50,wizardlm-70b,1
2,sharegpt,You will act as an voice changer. You will cha...,An umbrella has been brought by her on the way.,4.50,wizardlm-13b,1
3,ultrachat,Write a well-researched paper on the physiolog...,Title: The Physiological and Psychological Imp...,4.75,wizardlm-13b,1
4,ultrachat,Create a step-by-step recipe for making fluffy...,"Of course, I'd be happy to help you with that!...",5.00,llama-2-7b-chat,1
...,...,...,...,...,...,...
10203,sharegpt,Transform is an emerging brand that delivers m...,"Sure, here's a possible description for the ""A...",4.00,llama-2-70b-chat,0
10204,evol_instruct,"Using Python and the scikit-learn library, how...","Hello! As a helpful and respectful assistant, ...",4.00,llama-2-13b-chat,0
10205,flan_v2_cot,"[QUESTION] If ""Two kids stepping stones over t...",Confidence: 80%\n\nYou have provided several s...,1.75,llama-2-7b-chat,0
10206,sharegpt,Write me a letter asking my friend Dan what he...,I cannot fulfill your request as it goes again...,1.25,llama-2-7b-chat,0


In [420]:
# Putting metadata back in:
df_all = df_all.merge(df_clean.loc[:, ['prompt', 'x_cord', 'y_cord','topic_id', 'embeddings', 'metadata', 'topic_name']], on='prompt')

In [421]:
# mix them up 
df_all = df_all.sample(frac=1, random_state=42).reset_index(drop=True)

In [422]:
#for some reason these rows don't work, idk why. Your dataset might not run in to this issue
bads = [5230, 19157]

df_all = df_all.drop(bads)

## Getting statistics form each of the responses

In [423]:
#Get text statisitsics for each response 

# NB! This takes forever to run so try to run it only once

for i, row in df_all.iterrows():
    #Make each into a SpaCy doc

    response = make_spacy_doc(row['response'], lang='en_core_web_sm')
    
    #Basics

    df_all.loc[i, 'n_sents'] = text_stats.basics.n_sents(response)

    df_all.loc[i, 'n_words'] = text_stats.basics.n_words(response)

    df_all.loc[i, 'n_uniquewords'] = text_stats.basics.n_unique_words(response)

    df_all.loc[i, 'n_chars'] = text_stats.basics.n_chars(response)

    df_all.loc[i, 'n_longwords'] = text_stats.basics.n_long_words(response)

    df_all.loc[i, 'n_sylsprword'] = text_stats.basics.n_syllables(response)

    df_all.loc[i, 'n_monosylwords'] = text_stats.basics.n_monosyllable_words(response)

    df_all.loc[i, 'n_polysylwords'] = text_stats.basics.n_polysyllable_words(response)

    df_all.loc[i, 'entropy'] = text_stats.basics.entropy(response)

    #Lexical Diversity

    df_all.loc[i, 'ttr'] = text_stats.diversity.ttr(response)

    df_all.loc[i, 'log_ttr'] = text_stats.diversity.log_ttr(response)

    df_all.loc[i, 'seg_ttr'] = text_stats.diversity.segmented_ttr(response)

    df_all.loc[i, 'mtld'] = text_stats.diversity.mtld(response)

    df_all.loc[i, 'hdd'] = text_stats.diversity.hdd(response)
    
    #Readability

    df_all.loc[i, 'flesch_score'] = text_stats.readability.flesch_reading_ease(response) 

    df_all.loc[i, 'flesch_grade'] = text_stats.readability.flesch_kincaid_grade_level(response)

    df_all.loc[i, 'gunning_fog'] = text_stats.readability.gunning_fog_index(response) 

    df_all.loc[i, 'coleman_liau'] = text_stats.readability.coleman_liau_index(response) 

    df_all.loc[i, 'automated_readability'] = text_stats.readability.automated_readability_index(response)


In [424]:
# df_checkpoint  = df_all.copy()

In [425]:
df_all = df_checkpoint

In [ ]:
#reordering for easier indexing
cols = df_all.columns.tolist()
cols = ['source',
 'prompt',
 'response',
 'model',
 'chosen',
 'rating',
 'n_sents',
 'n_words',
 'n_uniquewords',
 'n_chars',
 'n_longwords',
 'n_sylsprword',
 'n_monosylwords',
 'n_polysylwords',
 'entropy',
 'ttr',
 'log_ttr',
 'seg_ttr',
 'mtld',
 'hdd',
 'flesch_score',
 'flesch_grade',
 'gunning_fog',
 'coleman_liau',
 'automated_readability',
 'x_cord',
 'y_cord',
 'topic_id',
 'embeddings',
 'metadata',
 'topic_name']

df_all = df_all[cols]

In [428]:
df_all
df_all.to_csv("Accepted_Rejected_with_scores.csv")

In [429]:
stats_only = df_all.iloc[:, 4:-6]

## Indicators of 'Accepted'

In [430]:
from scipy.stats import pointbiserialr

In [431]:
results = {}
continuous_vars = stats_only.columns[1:]
dummy_vars = [stats_only.columns[0]]

index = pd.MultiIndex.from_product([continuous_vars])

corr_df = pd.DataFrame(index=continuous_vars, columns=dummy_vars)

for dummy in dummy_vars:
    for continuous in continuous_vars:
        corr, p_value = pointbiserialr(stats_only[continuous], stats_only[dummy])
        results[f'{continuous} & {dummy}'] = (corr, p_value)
        corr_df.loc[continuous, dummy] = corr
        

corr_df = corr_df.apply(pd.to_numeric)

pvalues = []
for i in results.values():
    pvalues.append((i[1]))

corr_df['p_values'] = pvalues

corr_df = corr_df.rename(columns={'chosen': 'PB_Correlation'})

In [432]:
# Need a standardized way of assesing variance of metrics

def coefficient_of_variation(series):
    return series.std() / series.mean()

cv = stats_only.iloc[:, 1:].apply(coefficient_of_variation)

corr_df['coef_of_variation'] = cv

In [434]:
corr_df.sort_values(by=['PB_Correlation', 'coef_of_variation', 'p_values'], ascending=[False, False, True])

,PB_Correlation,p_values,coef_of_variation
rating,0.614510,0.000000e+00,0.243758
n_uniquewords,0.182092,1.064511e-151,0.756658
n_longwords,0.175959,1.251788e-141,1.012037
n_chars,0.174384,4.207479e-139,0.919935
n_sylsprword,0.172199,1.232727e-135,0.918244
n_polysylwords,0.166823,2.638746e-127,1.128188
n_words,0.165324,4.957085e-125,0.882359
n_sents,0.158025,2.860982e-114,0.959185
n_monosylwords,0.150066,4.091511e-103,0.873543
entropy,0.118253,1.792173e-64,0.296626


## Statistically Significant metrics for 'Accepted' 

In [435]:
corr_df[corr_df['p_values'] <= 0.05].sort_values(by=['PB_Correlation', 'coef_of_variation'], ascending=[False, False])

,PB_Correlation,p_values,coef_of_variation
rating,0.614510,0.000000e+00,0.243758
n_uniquewords,0.182092,1.064511e-151,0.756658
n_longwords,0.175959,1.251788e-141,1.012037
n_chars,0.174384,4.207479e-139,0.919935
n_sylsprword,0.172199,1.232727e-135,0.918244
n_polysylwords,0.166823,2.638746e-127,1.128188
n_words,0.165324,4.957085e-125,0.882359
n_sents,0.158025,2.860982e-114,0.959185
n_monosylwords,0.150066,4.091511e-103,0.873543
entropy,0.118253,1.792173e-64,0.296626


### Which topics have the highest entropy/lowest entropy? n_polysylwords, etc.

In [436]:
pd.DataFrame(df_all.groupby('topic_name')['n_uniquewords'].agg('mean')).sort_values(by='n_uniquewords', ascending=False)

,n_uniquewords
topic_name,
Healthy Lifestyle Exploration,161.350560
Environmental Impact Solutions,140.747352
Urban Exploration,138.836694
Marketing Strategy Platform,138.130298
Creative Writing,111.754293
Multi-dimensional Gaming,100.490716
Culinary Creations,99.736897
Childhood Stress and Health,91.819386
Cultural Expression,91.467339


### Which documents have the highest entropy/lowest entropy? n_polysylwords, etc.

In [437]:
df_example = df_all.sort_values(by=['n_uniquewords', 'topic_name'], ascending=[False, False])
df_example = df_example.groupby('topic_name').head(1)
df_example = df_example[['topic_name', 'n_uniquewords', 'response']]
df_example

,topic_name,n_uniquewords,response
1730,Creative Writing,424.0,Title: Crescendo of the Heart\n\nChapter One -...
8349,Environmental Impact Solutions,404.0,Title: The Impact of Artificial Intelligence o...
20090,Healthy Lifestyle Exploration,392.0,Title: Electronic Voting Systems: A Glimpse in...
2936,Culinary Creations,377.0,"What purpose does frosting serve, and how does..."
3702,Cultural Expression,373.0,**In the acrid vapors of a renewed and retched...
17937,Urban Exploration,368.0,Hello! I'm here to help you format your Markdo...
2075,Marketing Strategy Platform,361.0,Title: On-the-Spot Business Plan\n\nExecutive ...
2946,Multi-dimensional Gaming,360.0,Of course! I'm glad to help you with that. Her...
18392,Gaming Economics,345.0,"Title: ""Fortnite Battle Royale Beginner's Guid..."
4580,Analyzing Product Sentiment,314.0,Title: The North Water\nTrailer Link: https://...
